# Reimplementación con Cython

## Características de la instancia

Se utilizó una máquina **t2.2xlarge** en lugar de una m4.16xlarge porque la cuenta de educate de aws que utilizamos no tenía permiso para levantar instancias de tipo m4. Sin embargo, buscamos y encontramos que las t2 y las m4 eran muy similares entre ellas por lo que nos decidimos por utlilzar la más grande de las tipo t2. La máquina t2.2xlarge tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              1
Core(s) per socket:              8
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2300.193
BogoMIPS:                        4600.11
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       256 KiB
L1i cache:                       256 KiB
L2 cache:                        2 MiB
L3 cache:                        45 MiB
NUMA node0 CPU(s):               0-7
Vul

In [1]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.2.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 32GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits


In [2]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-159 5.4.0-1038-aws #40-Ubuntu SMP Fri Feb 5 23:50:40 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


## Cython: Iteración 0

Para utilizar Cython recordemos que son necesarios 3 archivos:

1.El código que será compilado en un archivo con extensión .pyx (escrito en Python)
    
2.Un archivo setup.py que contiene las instrucciones para llamar a Cython y se encarga de crear el módulo compilado.
  
3.El código escrito en Python que importará el módulo compilado.

### Creación del archivo con extensión .pyx
Copiamos y pegamos el código robustecido de la parte 1 de esta práctica 2.

In [17]:
%%file scripts/ffmax_0.pyx
class vertex:
    """
    A vertex in a network.
    Attributes:
        name (string): name or identifier of vertex
        source (bool): whether the vertex is a source vertex or not
        sink (bool): whether the vertex is a sink vertex or not
    """

    def __init__(self, name, source=False, sink=False):
        self.name = name
        self.source = source
        self.sink = sink

class edge:
    """
    An edge in a netwokt, going from one vertex to another
    Attributes:
        start (vertex): the edge comes out of this vertex
        end (vertex): the edge arrives at this vertex
        capacity (float): edge's maximum flow capacity
        flow (float): current flow in the edge
        returnEdge (pointer): return edge which is used in the residual graph
    """

    def __init__(self, start, end, capacity):
        self.start = start
        self.end = end
        self.capacity = capacity
        self.flow = 0
        self.returnEdge = None

class create_flow_network:
    """
    A flow network to which we want to find the maximum flow posible going
    from one vertex to another.
    Attributes:
       vertices (list): lists all of vertices in the graph
       network (dictionary): maps every vertex's name to all of the edges
                             coming out of the said vertex
    """
    def __init__(self):
        self.vertices = []
        self.network = {}

    def get_source(self):
        """
        Finds the source vertex in the list of vertices in the network.
        """
        for vertex in self.vertices:
            if vertex.source == True:
                return vertex

        return None

    def get_sink(self):
        """
        Finds the sink vertex in the list of vertices in the flow network.
        """
        for vertex in self.vertices:
            if vertex.sink == True:
                return vertex

        return None

    def get_vertex(self, name):
        """
        Takes a vertex name finds it in the lists of vertices of the
        object of class create_flow_network.
        Args:
            name (string): name or identifier of vertex
        Returns:
            vertex (vertex): object of class vertex corresponding to the
                             input vertex name.
        """
        for vertex in self.vertices:
            if name == vertex.name:
                return vertex

    def vertex_in_network(self, name):
        """
        Verifies if a certain vertex is in the list of vertices of the flow
        network.
        Args:
            name (string): name or identifier of vertex.
        Returns:
            (bool): if the vertex is in the network or not.
        """
        for vertex in self.vertices:
            if vertex.name == name:
                return True

        return False

    def get_edges(self):
        """
        Takes information from the network vertices and gets a list of all
        the edges going in and out of this vertices.
        Returns:
            allEdges (list): list of all vertices in the flow network.
        """
        allEdges = []
        for vertex in self.network:
            for edge in self.network[vertex]:
                allEdges.append(edge)

        return allEdges

    def create_vertex(self, name, source=False, sink=False):
        """
        Creates and adds a vertex to the network after it checks various
        error cases to ensure that the vertex can be added.
        Args:
            name (string): name or identifier of vertex_in_network
            source (bool): whether the vertex to add is source or not
            sink (bool): whether the vertex to add is sink or not
        Returns:
            (string): error message when error arises
        """
        if source == True and sink == True:
            return "El nodo {} no puede ser origen y destino".format(name)

        if self.vertex_in_network(name):
            return "Nodo duplicado"

        if source == True:
            if self.get_source() != None:
                return "Ya existe nodo origen"

        if sink == True:
            if self.get_sink() != None:
                return "Ya existe nodo destino"

        newVertex = vertex(name, source, sink)
        self.vertices.append(newVertex)
        self.network[newVertex.name] = []

    def create_edge(self, start, end, capacity):
        """
        Creates and adds a new edge to the flow network with capacity of 0
        by first checking the start and end vertices of said edge to
        verify that the are not the same vertex and that they are both in
        the network.
        Args:
            start (vertex): start vertex of the new edge
            end (vertex): end vertex of the new edge
            capacity (float): capcity of the new edge
        Returns:
            (string): error message when error arises
        """

        if self.vertex_in_network(start) == False:
            print("Nodo origen ya ha sido agregado. \n El cálculo de flujo máximo continúa con el primer valor asignado al nodo orígen.")

        elif self.vertex_in_network(end) == False:
            print("Nodo destino ya ha sido agregado. \n El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.")
        
        elif start == end:
            print("No se pueden tener bucles. \n El cálculo de flujo máximo continuará sin tomar en cuenta este arco.")
          
        else:
            newEdge = edge(start, end, capacity)
            returnEdge = edge(end, start, 0)
            newEdge.returnEdge = returnEdge
            returnEdge.returnEdge = newEdge
            vertex = self.get_vertex(start)
            self.network[vertex.name].append(newEdge)
            returnVertex = self.get_vertex(end)
            self.network[returnVertex.name].append(returnEdge)

    def get_path(self, start, end, path):
        """
        Recursive function that walks through the network starting at a certain
        vertex and calculates residual capacity for each edge it passes then
        uses this residual capacity to define how much flow to send along a
        given path. Then repeats this process until it reaches the end of the
        flow network.
        Args:
            start (vertex): start vertex of the new edge
            end (vertex): end vertex of the new edge
            path (list): list of vertices in a path
        Returns:
            path (list): list of vertices in a path
        """

        if start == end:
            return path

        for edge in self.network[start]:
            residualCapacity = edge.capacity - edge.flow
            if residualCapacity > 0 and not (edge, residualCapacity) in path:
                result = self.get_path(edge.end, end, path + [(edge, residualCapacity)])
                if result != None:
                    return result

    def MaxFlow(self):
        """
        Follows the path returned by get_path and gets the maximum flow in the
        network. This function implements the Ford Fulkerson method and
        calculates the flow while the path is not fully walked. It sums this
        flow to the fordward edges and substracts it from the reverse edges.
        Then, another path is calculated and we repeat the process.
        Returns:
            (string): error message when an error in the definition of the
                      network occurs.
            (float): maximum flow through the network
        """
        source = self.get_source()
        sink = self.get_sink()

        if source == None or sink == None:
            return "La red no tiene nodo origen y/o destino "

        path = self.get_path(source.name, sink.name, [])
        while path != None:
            flow = min(edge[1] for edge in path)
            for edge, res in path:
                edge.flow += flow
                edge.returnEdge.flow -= flow
            path = self.get_path(source.name, sink.name, [])
        return sum(edge.flow for edge in self.network[source.name])


Overwriting scripts/ffmax_0.pyx


### Creación del archivo setup.py

In [18]:
%%file scripts/setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("scripts/ffmax_0.pyx", 
                              compiler_directives={'language_level' : 3}))

Overwriting scripts/setup.py


### Compilación desde línea de comandos

In [19]:
%%bash
python3 scripts/setup.py build_ext --inplace

Compiling scripts/ffmax_0.pyx because it changed.
[1/1] Cythonizing scripts/ffmax_0.pyx
running build_ext
building 'ffmax_0' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c scripts/ffmax_0.c -o build/temp.linux-x86_64-3.8/scripts/ffmax_0.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/scripts/ffmax_0.o -o /home/ubuntu/practica-2-segunda-parte-diramtz/ffmax_0.cpython-38-x86_64-linux-gnu.so


### Importar y probar el módulo
Probamos el módulo con nuestro ejemplo base.

In [1]:
import ffmax_0 as ff
import time
import networkx as nx
from pytest import approx
from IPython.display import HTML, display

In [2]:
start_time = time.time()

red = ff.create_flow_network()

red.create_vertex('o', True, False) 
red.create_vertex('t', False, True) 
red.create_vertex('a', False, False)
red.create_vertex('b', False, False)
red.create_vertex('c', False, False)
red.create_vertex('d', False, False)
red.create_vertex('e', False, False)

red.create_edge('o', 'a', 5)
red.create_edge('o', 'b', 7)
red.create_edge('o', 'c', 4)

red.create_edge('a', 'b', 1)
red.create_edge('a', 'd', 3)

red.create_edge('b', 'c', 2)
red.create_edge('b', 'd', 4)
red.create_edge('b', 'e', 5)

red.create_edge('c', 'e', 4)

red.create_edge('d', 't', 9)

red.create_edge('e', 'd', 1)
red.create_edge('e', 't', 9)

red.MaxFlow()

end_time = time.time()

In [3]:
secs = end_time-start_time
print("El ejemplo base tomó",secs,"segundos" )

El ejemplo base tomó 0.0008416175842285156 segundos


Y verificamos que continuamos resolviendo el problema de manera correcta.

In [4]:
G = nx.DiGraph()

G.add_edge('o', 'a', capacity=5)
G.add_edge('o', 'b', capacity=7)
G.add_edge('o', 'c', capacity=4)

G.add_edge('a', 'b', capacity=1)
G.add_edge('a', 'd', capacity=3)

G.add_edge('b', 'c', capacity=2)
G.add_edge('b', 'd', capacity=4)
G.add_edge('b', 'e', capacity=5)

G.add_edge('c', 'e', capacity=4)

G.add_edge('d', 't', capacity=9)

G.add_edge('e', 'd', capacity=1)
G.add_edge('e', 't', capacity=9)

flow_value, flow_dict = nx.maximum_flow(G, 'o', 't')

In [5]:
flow_value

15

In [6]:
red.MaxFlow()

15

In [7]:
obj = flow_value
res = red.MaxFlow()
print(res == approx(obj))

True


### Análisis del archivo HTML

In [8]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate scripts/ffmax_0.pyx

In [9]:
display(HTML("scripts/ffmax_0.html"))

## Cython: Iteración 1

### Creación del archivo con extensión .pyx
Realizamos los cambios a las líneas que detectamos "son muy amarillas" en el HTML anterior y las reimplementamos con código en C.

Los cambios realizados fueron a las líneas:
+ 010
+ 109
+ 138

In [31]:
%%file scripts/ffmax_1.pyx
from libcpp cimport bool as bool_t
cdef class vertex:
    """
    A vertex in a network.
    Attributes:
        name (string): name or identifier of vertex
        source (bool): whether the vertex is a source vertex or not
        sink (bool): whether the vertex is a sink vertex or not
    """
    
    def __init__(self, name, bool_t source=False, bool_t sink=False):
        self.name = name
        self.source = source
        self.sink = sink

cdef class edge:
    """
    An edge in a netwokt, going from one vertex to another
    Attributes:
        start (vertex): the edge comes out of this vertex
        end (vertex): the edge arrives at this vertex
        capacity (float): edge's maximum flow capacity
        flow (float): current flow in the edge
        returnEdge (pointer): return edge which is used in the residual graph
    """
    
    cdef double flow, capacity
    
    def __init__(self, start, end, capacity):
        self.start = start
        self.end = end
        self.capacity = capacity
        self.flow = 0
        self.returnEdge = None

cdef class create_flow_network:
    """
    A flow network to which we want to find the maximum flow posible going
    from one vertex to another.
    Attributes:
       vertices (list): lists all of vertices in the graph
       network (dictionary): maps every vertex's name to all of the edges
                             coming out of the said vertex
    """
    
    cdef public list vertices
    
    def __init__(self):
        self.vertices = []
        self.network = {}

    def get_source(self):
        """
        Finds the source vertex in the list of vertices in the network.
        """
        for vertex in self.vertices:
            if vertex.source == True:
                return vertex

        return None

    def get_sink(self):
        """
        Finds the sink vertex in the list of vertices in the flow network.
        """
        for vertex in self.vertices:
            if vertex.sink == True:
                return vertex

        return None

    def get_vertex(self, name):
        """
        Takes a vertex name finds it in the lists of vertices of the
        object of class create_flow_network.
        Args:
            name (string): name or identifier of vertex
        Returns:
            vertex (vertex): object of class vertex corresponding to the
                             input vertex name.
        """
        for vertex in self.vertices:
            if name == vertex.name:
                return vertex

    def vertex_in_network(self, name):
        """
        Verifies if a certain vertex is in the list of vertices of the flow
        network.
        Args:
            name (string): name or identifier of vertex.
        Returns:
            (bool): if the vertex is in the network or not.
        """
        for vertex in self.vertices:
            if vertex.name == name:
                return True

        return False

    def get_edges(self):
        """
        Takes information from the network vertices and gets a list of all
        the edges going in and out of this vertices.
        Returns:
            allEdges (list): list of all vertices in the flow network.
        """
        allEdges = []
        for vertex in self.network:
            for edge in self.network[vertex]:
                allEdges.append(edge)

        return allEdges

    def create_vertex(self, name, bool_t source=False, bool_t sink=False):
        """
        Creates and adds a vertex to the network after it checks various
        error cases to ensure that the vertex can be added.
        Args:
            name (string): name or identifier of vertex_in_network
            source (bool): whether the vertex to add is source or not
            sink (bool): whether the vertex to add is sink or not
        Returns:
            (string): error message when error arises
        """
        if source == True and sink == True:
            return "El nodo {} no puede ser origen y destino".format(name)

        if self.vertex_in_network(name):
            return "Nodo duplicado"

        if source == True:
            if self.get_source() != None:
                return "Ya existe nodo origen"

        if sink == True:
            if self.get_sink() != None:
                return "Ya existe nodo destino"

        newVertex = vertex(name, source, sink)
        self.vertices.append(newVertex)
        self.network[newVertex.name] = []

    def create_edge(self, start, end, double capacity):
        """
        Creates and adds a new edge to the flow network with capacity of 0
        by first checking the start and end vertices of said edge to
        verify that the are not the same vertex and that they are both in
        the network.
        Args:
            start (vertex): start vertex of the new edge
            end (vertex): end vertex of the new edge
            capacity (float): capcity of the new edge
        Returns:
            (string): error message when error arises
        """

        if self.vertex_in_network(start) == False:
            print("Nodo origen ya ha sido agregado. \n El cálculo de flujo máximo continúa con el primer valor asignado al nodo orígen.")

        elif self.vertex_in_network(end) == False:
            print("Nodo destino ya ha sido agregado. \n El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.")
        
        elif start == end:
            print("No se pueden tener bucles. \n El cálculo de flujo máximo continuará sin tomar en cuenta este arco.")
          
        else:
            newEdge = edge(start, end, capacity)
            returnEdge = edge(end, start, 0)
            newEdge.returnEdge = returnEdge
            returnEdge.returnEdge = newEdge
            vertex = self.get_vertex(start)
            self.network[vertex.name].append(newEdge)
            returnVertex = self.get_vertex(end)
            self.network[returnVertex.name].append(returnEdge)

    def get_path(self, start, end, path):
        """
        Recursive function that walks through the network starting at a certain
        vertex and calculates residual capacity for each edge it passes then
        uses this residual capacity to define how much flow to send along a
        given path. Then repeats this process until it reaches the end of the
        flow network.
        Args:
            start (vertex): start vertex of the new edge
            end (vertex): end vertex of the new edge
            path (list): list of vertices in a path
        Returns:
            path (list): list of vertices in a path
        """

        if start == end:
            return path

        for edge in self.network[start]:
            residualCapacity = edge.capacity - edge.flow
            if residualCapacity > 0 and not (edge, residualCapacity) in path:
                result = self.get_path(edge.end, end, path + [(edge, residualCapacity)])
                if result != None:
                    return result

    def MaxFlow(self):
        """
        Follows the path returned by get_path and gets the maximum flow in the
        network. This function implements the Ford Fulkerson method and
        calculates the flow while the path is not fully walked. It sums this
        flow to the fordward edges and substracts it from the reverse edges.
        Then, another path is calculated and we repeat the process.
        Returns:
            (string): error message when an error in the definition of the
                      network occurs.
            (float): maximum flow through the network
        """
        cdef double s, flow
        
        source = self.get_source()
        sink = self.get_sink()

        if source == None or sink == None:
            return "La red no tiene nodo origen y/o destino "

        path = self.get_path(source.name, sink.name, [])
        while path != None:
            flow = min(edge[1] for edge in path)
            for edge, res in path:
                edge.flow += flow
                edge.returnEdge.flow -= flow
            path = self.get_path(source.name, sink.name, [])
        for edge in self.network[source.name]:
            s += edge.flow
        return s


Overwriting scripts/ffmax_1.pyx


### Análisis de archivo HTML

In [32]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate scripts/ffmax_1.pyx

In [33]:
display(HTML("scripts/ffmax_1.html"))

### Prueba de reimplementación

In [ ]:
# checar que sigue resolviendo el problema base

## Cython: Iteración 2

### Creación del archivo .pyx